In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.imports import *
from fastai.torch_imports import *
from fastai.core import *
from fastai.model import fit
from fastai.dataset import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle

In [2]:
bs,bptt = 64,70

## Language modeling

### Data

In [5]:
PATH='data/aclImdb/'

TRN_PATH = 'train/all/'
VAL_PATH = 'test/all/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

%ls {PATH}

aclImdb_v1.tar.gz  imdbEr.txt  imdb.vocab  models/  README  test/  tmp/  train/


In [5]:
%ls {TRN} | head

0_0.txt
0_3.txt
0_9.txt
10000_0.txt
10000_4.txt
10000_8.txt
1000_0.txt
10001_0.txt
10001_10.txt
10001_4.txt
ls: write error


In [73]:
!cat {TRN}1000_0.txt

I have to say when a name like Zombiegeddon and an atom bomb on the front cover I was expecting a flat out chop-socky fung-ku, but what I got instead was a comedy. So, it wasn't quite was I was expecting, but I really liked it anyway! The best scene ever was the main cop dude pulling those kids over and pulling a Bad Lieutenant on them!! I was laughing my ass off. I mean, the cops were just so bad! And when I say bad, I mean The Shield Vic Macky bad. But unlike that show I was laughing when they shot people and smoked dope.<br /><br />Felissa Rose...man, oh man. What can you say about that hottie. She was great and put those other actresses to shame. She should work more often!!!!! I also really liked the fight scene outside of the building. That was done really well. Lots of fighting and people getting their heads banged up. FUN! Last, but not least Joe Estevez and William Smith were great as the...well, I wasn't sure what they were, but they seemed to be having fun and throwing out l

In [74]:
!find {TRN} -name '*.txt' | xargs cat | wc -w

17486581


In [13]:
!find {VAL} -name '*.txt' | xargs cat | wc -w

5686719


In [7]:
TEXT = data.Field(lower=True, tokenize=spacy_tok)
FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

In [9]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(4602, 34945, 1, 20621966, 34945)

In [77]:
TEXT.vocab.itos[:12]

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is', 'it', 'in']

In [78]:
md.trn_ds[0].text[:12], next(iter(md.trn_dl))

(['i',
  'just',
  'do',
  "n't",
  'get',
  'some',
  'of',
  'the',
  'big',
  'premises',
  'of',
  'this'],
 (Variable containing:
      12    687     13  ...    1923     19     17
      51    615     25  ...      10     63     51
      53      3      3  ...      42    428    833
          ...            ⋱           ...         
      51     64      3  ...     111    544      6
       6      2   2075  ...      90     43    552
    3343   4351      5  ...      35      2    249
  [torch.cuda.LongTensor of size 61x64 (GPU 0)], Variable containing:
      51
     615
      25
    ⋮   
      93
     406
      43
  [torch.cuda.LongTensor of size 3904 (GPU 0)]))

### Train

In [6]:
em_sz = 200
nh = 500
nl = 3
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [9]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
               dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [11]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

A Jupyter Widget

[ 0.      4.6754  4.5553]                                     
[ 1.      4.5344  4.4255]                                     
[ 2.      4.4838  4.389 ]                                     
[ 3.      4.5597  4.4271]                                     
[ 4.      4.4565  4.3472]                                     
[ 5.      4.4095  4.3167]                                     
[ 6.      4.5065  4.3852]                                     
[ 7.      4.4273  4.3122]                                     
[ 8.      4.3651  4.2831]                                     



In [12]:
learner.save_encoder('adam1_enc')

In [13]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=10, cycle_save_name='adam3_10')

A Jupyter Widget

[ 0.      4.5019  4.3872]                                     
[ 1.      4.4707  4.3654]                                     
[ 2.      4.4606  4.3432]                                     
[ 3.      4.4183  4.3185]                                     
[ 4.      4.3858  4.2928]                                     
[ 5.      4.368   4.2717]                                     
[ 6.      4.3286  4.2485]                                     
[ 7.      4.3101  4.2333]                                     
[ 8.      4.2913  4.2256]                                     
[ 9.      4.2581  4.2244]                                     
[ 10.       4.4615   4.3373]                                  
[ 11.       4.4281   4.3265]                                  
[ 12.       4.4209   4.3076]                                  
[ 13.       4.3941   4.2883]                                  
[ 14.       4.3655   4.2645]                                  
[ 15.       4.3298   4.242 ]                           

In [14]:
learner.save_encoder('adam3_10_enc')

In [15]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=20, cycle_save_name='adam3_20')

A Jupyter Widget

[ 0.      4.2122  4.1803]                                     
[ 1.      4.4048  4.2924]                                     
[ 2.      4.4049  4.292 ]                                     
[ 3.      4.3769  4.2839]                                     
[ 4.      4.3734  4.2757]                                     
[ 5.      4.3558  4.2684]                                     
[ 6.      4.3471  4.2587]                                     
[ 7.      4.3372  4.2475]                                     
[ 8.      4.3426  4.2398]                                     
[ 9.      4.3155  4.2253]                                     
[ 10.       4.2975   4.2165]                                  
[ 11.       4.2747   4.2057]                                  
[ 12.       4.2725   4.1949]                                  
[ 13.       4.2533   4.1861]                                  
[ 14.       4.2281   4.1788]                                  
[ 15.       4.226    4.1729]                           

In [16]:
learner.save_encoder('adam3_20_enc')

In [17]:
math.exp(4.165)

64.3926824434624

### Test

In [18]:
m=learner.model
s=""". So, it wasn't quite was I was expecting, but I really liked it anyway! The best"""
s = [spacy_tok(s)]
t=TEXT.numericalize(s)
' '.join(s[0])

". So , it was n't quite was I was expecting , but I really liked it anyway ! The best"

In [19]:
m[0].bs=1
m.eval()
m.reset()
res,*_ = m(t)
m[0].bs=bs

In [20]:
nexts = torch.topk(res[-1], 10)[1]
[TEXT.vocab.itos[o] for o in to_np(nexts)]

['performance',
 'of',
 'friend',
 'actor',
 'thing',
 'scene',
 'character',
 'part',
 'line',
 'movie']

In [21]:
for i in range(20):
    n=res[-1].topk(2)[1]
    n = n[1] if n.data[0]==0 else n[0]
    print(TEXT.vocab.itos[n.data[0]], end=' ')
    res,*_ = m(n[0].unsqueeze(0))

performance was the one in the movie where he was a little too old for the part . i think 

### Sentiment

In [7]:
TEXT = pickle.load(open(f'{PATH}models/TEXT.pkl','rb'))

In [8]:
IMDB_LABEL = data.Field(sequential=False)
splits = torchtext.datasets.IMDB.splits(TEXT, IMDB_LABEL, 'data/')
md2 = TextData.from_splits(PATH, splits, bs)

In [9]:
# dropout=0.4, dropouth=0.3, dropouti=0.65, dropoute=0.1, wdrop=0.5
#            dropout=0.3, dropouti=0.4, wdrop=0.3, dropoute=0.05, dropouth=0.2)

In [103]:
m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl, 
           dropout=0.1, dropouti=0.4, wdrop=0.5, dropoute=0.05, dropouth=0.3)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.clip=25.
m3.load_encoder(f'adam3_20_enc')
lrs=np.array([1e-4,1e-3,1e-2])

In [104]:
m3.freeze_to(-1)
m3.fit(lrs/2, 1, metrics=[accuracy])
m3.unfreeze()
m3.fit(lrs, 1, metrics=[accuracy], cycle_len=1)

A Jupyter Widget

[ 0.      0.43    0.2399  0.9075]                            



A Jupyter Widget

[ 0.      0.3     0.1924  0.9272]                            



In [59]:
#   dropout=0.1, dropouti=0.4, wdrop=0.5, dropoute=0.05, dropouth=0.3)
m3.fit(lrs, 8, metrics=[accuracy], cycle_len=2, cycle_save_name='imdb2')

A Jupyter Widget

[ 0.      0.275   0.1847  0.9306]                            
[ 1.      0.2528  0.1781  0.9336]                            
[ 2.      0.2448  0.1729  0.9365]                            
[ 3.      0.2052  0.1721  0.9378]                            
[ 4.      0.2239  0.1619  0.9413]                            
[ 5.      0.1956  0.1628  0.9417]                            
[ 6.      0.2093  0.158   0.9421]                            
[ 7.      0.1733  0.1591  0.9432]                            
[ 8.      0.1916  0.1552  0.9443]                            
[ 9.      0.1652  0.157   0.9451]                            
[ 10.       0.1749   0.1609   0.9427]                        
[ 11.       0.147    0.1579   0.9441]                        
[ 12.       0.1554   0.1626   0.942 ]                        
[ 13.       0.1302   0.1603   0.9437]                        
[ 14.       0.1346   0.1802   0.9384]                        
[ 15.       0.1425   0.1711   0.941 ]                        



In [78]:
m3.load_cycle('imdb2', 4)

In [79]:
accuracy(*m3.predict_with_targs())

0.94511217948717952

In [ ]:
def pred_snapshot(learner, name, cycle):
    learner.load_cycle(name, cycle)
    return learner.predict_with_targs()

cyc_preds = [pred_snapshot(m3, 'imdb2', i) for i in range(4,7)]

In [15]:
preds = [p[0] for p in cyc_preds[:]]
y = cyc_preds[0][1]
print(accuracy(np.mean(preds,0),y))

### End

- Aux task, not just pretrain
- Attention, not just average pool
- Pretrain language model on wikitext-103
- Bidi RNN

In [20]:
import sys
import ctypes

CUDA_SUCCESS = 0
CU_DEVICE_ATTRIBUTE_MULTIPROCESSOR_COUNT = 16
CU_DEVICE_ATTRIBUTE_MAX_THREADS_PER_MULTIPROCESSOR = 39
CU_DEVICE_ATTRIBUTE_CLOCK_RATE = 13
CU_DEVICE_ATTRIBUTE_MEMORY_CLOCK_RATE = 36

libnames = ('libcuda.so', 'libcuda.dylib', 'cuda.dll')
for libname in libnames:
    try:
        cuda = ctypes.CDLL(libname)
    except OSError:
        continue
    else:
        break
else:
    raise OSError("could not load any of: " + ' '.join(libnames))

nGpus = ctypes.c_int()
name = b' ' * 100
freeMem = ctypes.c_size_t()
totalMem = ctypes.c_size_t()

result = ctypes.c_int()
device = ctypes.c_int()
context = ctypes.c_void_p()
error_str = ctypes.c_char_p()

result = cuda.cuInit(0)
if result != CUDA_SUCCESS:
    cuda.cuGetErrorString(result, ctypes.byref(error_str))
    print("cuInit failed with error code %d: %s" % (result, error_str.value.decode()))
result = cuda.cuDeviceGetCount(ctypes.byref(nGpus))
if result != CUDA_SUCCESS:
    cuda.cuGetErrorString(result, ctypes.byref(error_str))
    print("cuDeviceGetCount failed with error code %d: %s" % (result, error_str.value.decode()))
print("Found %d device(s)." % nGpus.value)

i=0
result = cuda.cuDeviceGet(ctypes.byref(device), i)
if result != CUDA_SUCCESS:
    cuda.cuGetErrorString(result, ctypes.byref(error_str))
    print("cuDeviceGet failed with error code %d: %s" % (result, error_str.value.decode()))
print("Device: %d" % i)
if cuda.cuDeviceGetName(ctypes.c_char_p(name), len(name), device) == CUDA_SUCCESS:
    print("  Name: %s" % (name.split(b'\0', 1)[0].decode()))
result = cuda.cuCtxCreate(ctypes.byref(context), 0, device)
if result != CUDA_SUCCESS:
    cuda.cuGetErrorString(result, ctypes.byref(error_str))
    print("cuCtxCreate failed with error code %d: %s" % (result, error_str.value.decode()))
else:
    result = cuda.cuMemGetInfo(ctypes.byref(freeMem), ctypes.byref(totalMem))
    if result == CUDA_SUCCESS:
        print("  Total Memory: %ld MiB" % (totalMem.value / 1024**2))
        print("  Free Memory: %ld MiB" % (freeMem.value / 1024**2))
    else:
        cuda.cuGetErrorString(result, ctypes.byref(error_str))
        print("cuMemGetInfo failed with error code %d: %s" % (result, error_str.value.decode()))
    cuda.cuCtxDetach(context)

In [43]:
import pycuda.autoinit
import pycuda.driver as cuda

(free,total)=cuda.mem_get_info()
free/(1024**2),total/(1024**2)

(10386.25, 11172.375)